In [1]:
import json
from pathlib import Path

# Chargement
with open("output/unified_profiles.json", "r", encoding="utf-8") as f:
    unified_pairs = json.load(f)  # résultat du matching par paires

with open("output/unified_triplets.json", "r", encoding="utf-8") as f:
    unified_triplets = json.load(f)  # résultat du matching triplet

In [1]:
# evaluate_matching_strategies.py
import json
from pathlib import Path

MOROCCAN_CITIES = {"casablanca", "rabat", "mohammedia", "marrakech", "fes", "agadir", "tanger", "meknes", "oujda", "kenitra", "safi", "tetouan"}

def is_moroccan(profile):
    loc = str(profile.get("location", "")).lower()
    return ("morocco" in loc or "maroc" in loc or
            any(city in loc for city in MOROCCAN_CITIES))

def count_platforms(profiles):
    return {p["platform"] for p in profiles}

def evaluate_unified(unified_list, name):
    total_identities = len(unified_list)
    total_profiles = sum(len(u["profiles"]) for u in unified_list)
    pairs_count = 0
    triplets_count = 0
    moroccan_identities = 0
    high_confidence = 0  # ≥2 profils avec email commun ou score élevé

    all_pairs = []

    for u in unified_list:
        profiles = u["profiles"]
        platforms = count_platforms(profiles)
        n = len(profiles)

        # Compter les paires (combinaisons 2 à 2)
        pairs_in_cluster = n * (n - 1) // 2
        pairs_count += pairs_in_cluster

        if n >= 3 and len(platforms) == 3:
            triplets_count += 1
        elif n == 2 and len(platforms) == 2:
            pass  # paire classique

        # Vérifier si marocain
        if any(is_moroccan(p) for p in profiles):
            moroccan_identities += 1

        # Confiance : email partagé ou score ≥ 0.7
        emails = {p.get("email_norm") or p.get("email") for p in profiles}
        emails = {e for e in emails if e and "@" in str(e)}
        if len(emails) == 1 or u.get("score", 0) >= 0.7:
            high_confidence += 1

        # Stocker les paires pour analyse fine (optionnel)
        for i in range(len(profiles)):
            for j in range(i+1, len(profiles)):
                p1, p2 = profiles[i], profiles[j]
                all_pairs.append((p1, p2))

    return {
        "méthode": name,
        "identités_unifiées": total_identities,
        "profils_couverts": total_profiles,
        "paires_liées": pairs_count,
        "triplets_complets": triplets_count,
        "identités_marocaines": moroccan_identities,
        "confiance_élevée": high_confidence,
        "couverture_%": round(total_profiles / 11399 * 100, 1) if name == "Paires" else round(total_profiles / 11399 * 100, 1)
    }

def main():
    # Total de profils : 3770 + 4276 + 3353 = 11399
    TOTAL_PROFILES = 3770 + 4276 + 3353

    # Charger les deux résultats
    try:
        with open("output/unified_profiles.json", "r", encoding="utf-8") as f:
            unified_pairs = json.load(f)
    except FileNotFoundError:
        print("⚠️ unified_profiles.json manquant")
        unified_pairs = []

    try:
        with open("output/unified_triplets.json", "r", encoding="utf-8") as f:
            unified_triplets = json.load(f)
    except FileNotFoundError:
        print("⚠️ unified_triplets.json manquant")
        unified_triplets = []

    # Évaluer
    eval_pairs = evaluate_unified(unified_pairs, "Paires (transitivité)")
    eval_triplets = evaluate_unified(unified_triplets, "Triplets (simultanés)")

    # Afficher comparaison
    print("\n" + "="*80)
    print("📊 COMPARAISON DES STRATÉGIES DE MATCHING")
    print("="*80)
    print(f"{'Métrique':<25} {'Paires (transitivité)':<25} {'Triplets (simultanés)':<25}")
    print("-"*80)

    keys = [
        "identités_unifiées",
        "profils_couverts",
        "paires_liées",
        "triplets_complets",
        "identités_marocaines",
        "confiance_élevée",
    ]

    for key in keys:
        val1 = eval_pairs[key]
        val2 = eval_triplets[key]
        print(f"{key.replace('_', ' ').title():<25} {val1:<25} {val2:<25}")

    print(f"\n💡 Interprétation :")
    print(f" → La méthode par paires maximise la couverture (rappel).")
    print(f" → La méthode triplet maximise la précision (qualité).")
    print(f" → Pour une analyse fine (ex: localisation), les triplets sont plus fiables.")
    print(f" → Pour une base exhaustive (ex: recrutement), les paires sont préférables.")

if __name__ == "__main__":
    main()


📊 COMPARAISON DES STRATÉGIES DE MATCHING
Métrique                  Paires (transitivité)     Triplets (simultanés)    
--------------------------------------------------------------------------------
Identités Unifiées        564                       388                      
Profils Couverts          4685                      1164                     
Paires Liées              196227                    1164                     
Triplets Complets         138                       388                      
Identités Marocaines      536                       385                      
Confiance Élevée          152                       386                      

💡 Interprétation :
 → La méthode par paires maximise la couverture (rappel).
 → La méthode triplet maximise la précision (qualité).
 → Pour une analyse fine (ex: localisation), les triplets sont plus fiables.
 → Pour une base exhaustive (ex: recrutement), les paires sont préférables.
